## LangGraph Open Deep Research - Supervisor-Researcher Architecture

In this notebook, we'll explore the **supervisor-researcher delegation architecture** for conducting deep research with LangGraph.

You can visit this repository to see the original application: [Open Deep Research](https://github.com/langchain-ai/open_deep_research)

Let's jump in!

## What We're Building

This implementation uses a **hierarchical delegation pattern** where:

1. **User Clarification** - Optionally asks clarifying questions to understand the research scope
2. **Research Brief Generation** - Transforms user messages into a structured research brief
3. **Supervisor** - A lead researcher that analyzes the brief and delegates research tasks
4. **Parallel Researchers** - Multiple sub-agents that conduct focused research simultaneously
5. **Research Compression** - Each researcher synthesizes their findings
6. **Final Report** - All findings are combined into a comprehensive report

![Architecture Diagram](https://i.imgur.com/Q8HEZn0.png)

This differs from a section-based approach by allowing dynamic task decomposition based on the research question, rather than predefined sections.

---

# 🤝 Breakout Room #1
## Deep Research Foundations

In this breakout room, we'll understand the architecture and components of the Open Deep Research system.

## Task 1: Dependencies

You'll need API keys for Anthropic (for the LLM) and Tavily (for web search). We'll configure the system to use Anthropic's Claude Sonnet 4 exclusively.

In [1]:
import os
import getpass
# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

True

## Task 2: State Definitions

The state structure is hierarchical with three levels:

### Agent State (Top Level)
Contains the overall conversation messages, research brief, accumulated notes, and final report.

### Supervisor State (Middle Level)
Manages the research supervisor's messages, research iterations, and coordinating parallel researchers.

### Researcher State (Bottom Level)
Each individual researcher has their own message history, tool call iterations, and research findings.

We also have structured outputs for tool calling:
- **ConductResearch** - Tool for supervisor to delegate research to a sub-agent
- **ResearchComplete** - Tool to signal research phase is done
- **ClarifyWithUser** - Structured output for asking clarifying questions
- **ResearchQuestion** - Structured output for the research brief

Let's import these from our library: [`open_deep_library/state.py`](open_deep_library/state.py)

In [2]:
# Import state definitions from the library
from open_deep_library.state import (
    # Main workflow states
    AgentState,           # Lines 65-72: Top-level agent state with messages, research_brief, notes, final_report
    AgentInputState,      # Lines 62-63: Input state is just messages
    
    # Supervisor states
    SupervisorState,      # Lines 74-81: Supervisor manages research delegation and iterations
    
    # Researcher states
    ResearcherState,      # Lines 83-90: Individual researcher with messages and tool iterations
    ResearcherOutputState, # Lines 92-96: Output from researcher (compressed research + raw notes)
    
    # Structured outputs for tool calling
    ConductResearch,      # Lines 15-19: Tool for delegating research to sub-agents
    ResearchComplete,     # Lines 21-22: Tool to signal research completion
    ClarifyWithUser,      # Lines 30-41: Structured output for user clarification
    ResearchQuestion,     # Lines 43-48: Structured output for research brief
)

## Task 3: Utility Functions and Tools

The system uses several key utilities:

### Search Tools
- **tavily_search** - Async web search with automatic summarization to stay within token limits
- Supports Anthropic native web search and Tavily API

### Reflection Tools
- **think_tool** - Allows researchers to reflect on their progress and plan next steps (ReAct pattern)

### Helper Utilities
- **get_all_tools** - Assembles the complete toolkit (search + MCP + reflection)
- **get_today_str** - Provides current date context for research
- Token limit handling utilities for graceful degradation

These are defined in [`open_deep_library/utils.py`](open_deep_library/utils.py)

In [3]:
# Import utility functions and tools from the library
from open_deep_library.utils import (
    # Search tool - Lines 43-136: Tavily search with automatic summarization
    tavily_search,
    
    # Reflection tool - Lines 219-244: Strategic thinking tool for ReAct pattern
    think_tool,
    
    # Tool assembly - Lines 569-597: Get all configured tools
    get_all_tools,
    
    # Date utility - Lines 872-879: Get formatted current date
    get_today_str,
    
    # Supporting utilities for error handling
    get_api_key_for_model,          # Lines 892-914: Get API keys from config or env
    is_token_limit_exceeded,         # Lines 665-701: Detect token limit errors
    get_model_token_limit,           # Lines 831-846: Look up model's token limit
    remove_up_to_last_ai_message,    # Lines 848-866: Truncate messages for retry
    anthropic_websearch_called,      # Lines 607-637: Detect Anthropic native search usage
    openai_websearch_called,         # Lines 639-658: Detect OpenAI native search usage
    get_notes_from_tool_calls,       # Lines 599-601: Extract notes from tool messages
)

## Task 4: Configuration System

The configuration system controls:

### Research Behavior
- **allow_clarification** - Whether to ask clarifying questions before research
- **max_concurrent_research_units** - How many parallel researchers can run (default: 5)
- **max_researcher_iterations** - How many times supervisor can delegate research (default: 6)
- **max_react_tool_calls** - Tool call limit per researcher (default: 10)

### Model Configuration
- **research_model** - Model for research and supervision (we'll use Anthropic)
- **compression_model** - Model for synthesizing findings
- **final_report_model** - Model for writing the final report
- **summarization_model** - Model for summarizing web search results

### Search Configuration
- **search_api** - Which search API to use (ANTHROPIC, TAVILY, or NONE)
- **max_content_length** - Character limit before summarization

Defined in [`open_deep_library/configuration.py`](open_deep_library/configuration.py)

In [4]:
# Import configuration from the library
from open_deep_library.configuration import (
    Configuration,    # Lines 38-247: Main configuration class with all settings
    SearchAPI,        # Lines 11-17: Enum for search API options (ANTHROPIC, TAVILY, NONE)
)

## Task 5: Prompt Templates

The system uses carefully engineered prompts for each phase:

### Phase 1: Clarification
**clarify_with_user_instructions** - Analyzes if the research scope is clear or needs clarification

### Phase 2: Research Brief
**transform_messages_into_research_topic_prompt** - Converts user messages into a detailed research brief

### Phase 3: Supervisor
**lead_researcher_prompt** - System prompt for the supervisor that manages delegation strategy

### Phase 4: Researcher
**research_system_prompt** - System prompt for individual researchers conducting focused research

### Phase 5: Compression
**compress_research_system_prompt** - Prompt for synthesizing research findings without losing information

### Phase 6: Final Report
**final_report_generation_prompt** - Comprehensive prompt for writing the final report

All prompts are defined in [`open_deep_library/prompts.py`](open_deep_library/prompts.py)

In [5]:
# Import prompt templates from the library
from open_deep_library.prompts import (
    clarify_with_user_instructions,                    # Lines 3-41: Ask clarifying questions
    transform_messages_into_research_topic_prompt,     # Lines 44-77: Generate research brief
    lead_researcher_prompt,                            # Lines 79-136: Supervisor system prompt
    research_system_prompt,                            # Lines 138-183: Researcher system prompt
    compress_research_system_prompt,                   # Lines 186-222: Research compression prompt
    final_report_generation_prompt,                    # Lines 228-308: Final report generation
)

## ❓ Question #1:

Explain the interrelationships between the three states (Agent, Supervisor, Researcher). Why don't we just make a single huge state?

##### Answer:

The interrelationship is hierarchical:
- Agent State (Top Level): Holds the global context (user interaction, research brief, final report) and orchestrates the overall workflow
- Supervisor State (Middle Level): Manages the delegation strategy. It receives the brief from the Agent State and spawns multiple Researchers
- Researcher State (Bottom Level): Independent, isolated states for each parallel subagent to conduct specific research and tool usage

Why not a single huge state?
- Context Window / Token Limits: A single state would accumulate every raw search result, tool output, and thought from every researcher. This would rapidly exceed the model's context window. Separate states allow researchers to do their detailed work, compress the results, and pass only the summary back up the chain
- Parallelism: Independent states allow multiple researchers to run simultaneously without their message histories collide or confusing the model with irrelevant context from other active researchers

## ❓ Question #2:

What are the advantages and disadvantages of importing these components instead of including them in the notebook?

##### Answer:
Advantages:
- Keeps the notebook clean: By moving the heavy lifting (like the state definitions and graph construction) into external files, the notebook itself is much easier to read and focus on the high-level workflow. It doesn't get cluttered with hundreds of lines of implementation code
- Reusability: It feels more like real software project. If I wanted to build a script or an app using this researcher later, I could just import open_deep_library instead of having to copy-paste giant code cells from a notebook

Disadvantages:
- Harder to learn the details: To understand exactly how something like the supervisor node works, I have to constantly context-switch and open the .py files instead of just scrolling up to see the code definition right there in a cell
- Slower experimentation: If I want to tweak a prompt or change a tool's logic to see what happens, I can't just edit a cell and hit run. I have to edit the file, save it, and often restart the kernel to get the changes to reload

## 🏗️ Activity #1: Explore the Prompts

Open `open_deep_library/prompts.py` and examine one of the prompt templates in detail.

**Requirements:**
1. Choose one prompt template (clarify, brief, supervisor, researcher, compression, or final report)
2. Explain what the prompt is designed to accomplish
3. Identify 2-3 key techniques used in the prompt (e.g., structured output, role definition, examples)
4. Suggest one improvement you might make to the prompt

**YOUR CODE HERE** - Write your analysis in a markdown cell below

I have analyzed the lead_researcher_prompt (lines 79-136).

1. Selected Prompt
lead_researcher_prompt (The Supervisor)

2. What it is designed to accomplish
This prompt orchestrates the "brain" of the operation. Instead of trying to answer the question directly, it acts as a manager with the following goals:

- Analyze the complexity of the user's request
- Decompose the problem into parallelizable chunks (if necessary)
- Delegate specific research tasks to sub-agents via the ConductResearch tool
- Aggregate results and decide when sufficient information has been found to stop

3. Key techniques used
- Role prompting: It explicitly defines the persona: "Think like a research manager with limited time and resources." This discourages the model from going down rabbit holes and encourages efficiency.
- Explicit Constraints: It sets firm boundaries to prevent infinite loops and token waste, such as "Limit tool calls... always stop after {max_researcher_iterations}" and "Bias towards single agent"
- Sequential Reasoning (CoT Enforcement): It forces a "Think, then Act" workflow by mandating the use of the think_tool before delegating and after receiving results. This forces the model to generate a chain-of-thought trace in the context window before committing to an action

4. Suggested Improvement
Add a "Cross-Check" step. Currently, the supervisor delegates tasks and then checks if it has "enough" info. It might be beneficial to explicitly instruct the supervisor to check for conflicting information between sub-agents. We can add a step in the <Show Your Thinking> section: "If multiple researchers return conflicting data (e.g., different dates for the same event), explicitly formulate a new query to resolve the conflict before marking research as complete."

---

# 🤝 Breakout Room #2
## Building & Running the Researcher

In this breakout room, we'll explore the node functions, build the graph, and run wellness research.

## Task 6: Node Functions - The Building Blocks

Now let's look at the node functions that make up our graph. We'll import them from the library and understand what each does.

### The Complete Research Workflow

The workflow consists of 8 key nodes organized into 3 subgraphs:

1. **Main Graph Nodes:**
   - `clarify_with_user` - Entry point that checks if clarification is needed
   - `write_research_brief` - Transforms user input into structured research brief
   - `final_report_generation` - Synthesizes all research into final report

2. **Supervisor Subgraph Nodes:**
   - `supervisor` - Lead researcher that plans and delegates
   - `supervisor_tools` - Executes supervisor's tool calls (delegation, reflection)

3. **Researcher Subgraph Nodes:**
   - `researcher` - Individual researcher conducting focused research
   - `researcher_tools` - Executes researcher's tool calls (search, reflection)
   - `compress_research` - Synthesizes researcher's findings

All nodes are defined in [`open_deep_library/deep_researcher.py`](open_deep_library/deep_researcher.py)

### Node 1: clarify_with_user

**Purpose:** Analyzes user messages and asks clarifying questions if the research scope is unclear.

**Key Steps:**
1. Check if clarification is enabled in configuration
2. Use structured output to analyze if clarification is needed
3. If needed, end with a clarifying question for the user
4. If not needed, proceed to research brief with verification message

**Implementation:** [`open_deep_library/deep_researcher.py` lines 60-115](open_deep_library/deep_researcher.py#L60-L115)

In [6]:
# Import the clarify_with_user node
from open_deep_library.deep_researcher import clarify_with_user

### Node 2: write_research_brief

**Purpose:** Transforms user messages into a structured research brief for the supervisor.

**Key Steps:**
1. Use structured output to generate detailed research brief from messages
2. Initialize supervisor with system prompt and research brief
3. Set up supervisor messages with proper context

**Why this matters:** A well-structured research brief helps the supervisor make better delegation decisions.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 118-175](open_deep_library/deep_researcher.py#L118-L175)

In [7]:
# Import the write_research_brief node
from open_deep_library.deep_researcher import write_research_brief

### Node 3: supervisor

**Purpose:** Lead research supervisor that plans research strategy and delegates to sub-researchers.

**Key Steps:**
1. Configure model with three tools:
   - `ConductResearch` - Delegate research to a sub-agent
   - `ResearchComplete` - Signal that research is done
   - `think_tool` - Strategic reflection before decisions
2. Generate response based on current context
3. Increment research iteration count
4. Proceed to tool execution

**Decision Making:** The supervisor uses `think_tool` to reflect before delegating research, ensuring thoughtful decomposition of the research question.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 178-223](open_deep_library/deep_researcher.py#L178-L223)

In [8]:
# Import the supervisor node (from supervisor subgraph)
from open_deep_library.deep_researcher import supervisor

### Node 4: supervisor_tools

**Purpose:** Executes the supervisor's tool calls, including strategic thinking and research delegation.

**Key Steps:**
1. Check exit conditions:
   - Exceeded maximum iterations
   - No tool calls made
   - `ResearchComplete` called
2. Process `think_tool` calls for strategic reflection
3. Execute `ConductResearch` calls in parallel:
   - Spawn researcher subgraphs for each delegation
   - Limit to `max_concurrent_research_units` (default: 5)
   - Gather all results asynchronously
4. Aggregate findings and return to supervisor

**Parallel Execution:** This is where the magic happens - multiple researchers work simultaneously on different aspects of the research question.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 225-349](open_deep_library/deep_researcher.py#L225-L349)

In [9]:
# Import the supervisor_tools node
from open_deep_library.deep_researcher import supervisor_tools

### Node 5: researcher

**Purpose:** Individual researcher that conducts focused research on a specific topic.

**Key Steps:**
1. Load all available tools (search, MCP, reflection)
2. Configure model with tools and researcher system prompt
3. Generate response with tool calls
4. Increment tool call iteration count

**ReAct Pattern:** Researchers use `think_tool` to reflect after each search, deciding whether to continue or provide their answer.

**Available Tools:**
- Search tools (Tavily or Anthropic native search)
- `think_tool` for strategic reflection
- `ResearchComplete` to signal completion
- MCP tools (if configured)

**Implementation:** [`open_deep_library/deep_researcher.py` lines 365-424](open_deep_library/deep_researcher.py#L365-L424)

In [10]:
# Import the researcher node (from researcher subgraph)
from open_deep_library.deep_researcher import researcher

### Node 6: researcher_tools

**Purpose:** Executes the researcher's tool calls, including searches and strategic reflection.

**Key Steps:**
1. Check early exit conditions (no tool calls, native search used)
2. Execute all tool calls in parallel:
   - Search tools fetch and summarize web content
   - `think_tool` records strategic reflections
   - MCP tools execute external integrations
3. Check late exit conditions:
   - Exceeded `max_react_tool_calls` (default: 10)
   - `ResearchComplete` called
4. Continue research loop or proceed to compression

**Error Handling:** Safely handles tool execution errors and continues with available results.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 435-509](open_deep_library/deep_researcher.py#L435-L509)

In [11]:
# Import the researcher_tools node
from open_deep_library.deep_researcher import researcher_tools

### Node 7: compress_research

**Purpose:** Compresses and synthesizes research findings into a concise, structured summary.

**Key Steps:**
1. Configure compression model
2. Add compression instruction to messages
3. Attempt compression with retry logic:
   - If token limit exceeded, remove older messages
   - Retry up to 3 times
4. Extract raw notes from tool and AI messages
5. Return compressed research and raw notes

**Why Compression?** Researchers may accumulate lots of tool outputs and reflections. Compression ensures:
- All important information is preserved
- Redundant information is deduplicated
- Content stays within token limits for the final report

**Token Limit Handling:** Gracefully handles token limit errors by progressively truncating messages.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 511-585](open_deep_library/deep_researcher.py#L511-L585)

In [12]:
# Import the compress_research node
from open_deep_library.deep_researcher import compress_research

### Node 8: final_report_generation

**Purpose:** Generates the final comprehensive research report from all collected findings.

**Key Steps:**
1. Extract all notes from completed research
2. Configure final report model
3. Attempt report generation with retry logic:
   - If token limit exceeded, truncate findings by 10%
   - Retry up to 3 times
4. Return final report or error message

**Token Limit Strategy:**
- First retry: Use model's token limit × 4 as character limit
- Subsequent retries: Reduce by 10% each time
- Graceful degradation with helpful error messages

**Report Quality:** The prompt guides the model to create well-structured reports with:
- Proper headings and sections
- Inline citations
- Comprehensive coverage of all findings
- Sources section at the end

**Implementation:** [`open_deep_library/deep_researcher.py` lines 607-697](open_deep_library/deep_researcher.py#L607-L697)

In [13]:
# Import the final_report_generation node
from open_deep_library.deep_researcher import final_report_generation

## Task 7: Graph Construction - Putting It All Together

The system is organized into three interconnected graphs:

### 1. Researcher Subgraph (Bottom Level)
Handles individual focused research on a specific topic:
```
START → researcher → researcher_tools → compress_research → END
               ↑            ↓
               └────────────┘ (loops until max iterations or ResearchComplete)
```

### 2. Supervisor Subgraph (Middle Level)
Manages research delegation and coordination:
```
START → supervisor → supervisor_tools → END
            ↑              ↓
            └──────────────┘ (loops until max iterations or ResearchComplete)
            
supervisor_tools spawns multiple researcher_subgraphs in parallel
```

### 3. Main Deep Researcher Graph (Top Level)
Orchestrates the complete research workflow:
```
START → clarify_with_user → write_research_brief → research_supervisor → final_report_generation → END
                 ↓                                       (supervisor_subgraph)
               (may end early if clarification needed)
```

Let's import the compiled graphs from the library.

In [14]:
# Import the pre-compiled graphs from the library
from open_deep_library.deep_researcher import (
    # Bottom level: Individual researcher workflow
    researcher_subgraph,    # Lines 588-605: researcher → researcher_tools → compress_research
    
    # Middle level: Supervisor coordination
    supervisor_subgraph,    # Lines 351-363: supervisor → supervisor_tools (spawns researchers)
    
    # Top level: Complete research workflow
    deep_researcher,        # Lines 699-719: Main graph with all phases
)

## Why This Architecture?

### Advantages of Supervisor-Researcher Delegation

1. **Dynamic Task Decomposition**
   - Unlike section-based approaches with predefined structure, the supervisor can break down research based on the actual question
   - Adapts to different types of research (comparisons, lists, deep dives, etc.)

2. **Parallel Execution**
   - Multiple researchers work simultaneously on different aspects
   - Much faster than sequential section processing
   - Configurable parallelism (1-20 concurrent researchers)

3. **ReAct Pattern for Quality**
   - Researchers use `think_tool` to reflect after each search
   - Prevents excessive searching and improves search quality
   - Natural stopping conditions based on information sufficiency

4. **Flexible Tool Integration**
   - Easy to add MCP tools for specialized research
   - Supports multiple search APIs (Anthropic, Tavily)
   - Each researcher can use different tool combinations

5. **Graceful Token Limit Handling**
   - Compression prevents token overflow
   - Progressive truncation in final report generation
   - Research can scale to arbitrary depths

### Trade-offs

- **Complexity:** More moving parts than section-based approach
- **Cost:** Parallel researchers use more tokens (but faster)
- **Unpredictability:** Research structure emerges dynamically

## Task 8: Running the Deep Researcher

Now let's see the system in action! We'll use it to research wellness strategies for improving sleep quality.

### Setup

We need to:
1. Set up the wellness research request
2. Configure the execution with Anthropic settings
3. Run the research workflow

In [15]:
# Set up the graph with Anthropic configuration
from IPython.display import Markdown, display
import uuid

# Note: deep_researcher is already compiled from the library
# For this demo, we'll use it directly without additional checkpointing
graph = deep_researcher

print("✓ Graph ready for execution")
print("  (Note: The graph is pre-compiled from the library)")

✓ Graph ready for execution
  (Note: The graph is pre-compiled from the library)


### Configuration for Anthropic

We'll configure the system to use:
- **Claude Sonnet 4** for all research, supervision, and report generation
- **Tavily** for web search (you can also use Anthropic's native search)
- **Moderate parallelism** (1 concurrent researcher for cost control)
- **Clarification enabled** (will ask if research scope is unclear)

In [16]:
# Configure for Anthropic with moderate settings
config = {
    "configurable": {
        # Model configuration - using Claude Sonnet 4 for everything
        "research_model": "anthropic:claude-sonnet-4-20250514",
        "research_model_max_tokens": 10000,
        
        "compression_model": "anthropic:claude-sonnet-4-20250514",
        "compression_model_max_tokens": 8192,
        
        "final_report_model": "anthropic:claude-sonnet-4-20250514",
        "final_report_model_max_tokens": 10000,
        
        "summarization_model": "anthropic:claude-sonnet-4-20250514",
        "summarization_model_max_tokens": 8192,
        
        # Research behavior
        "allow_clarification": True,
        "max_concurrent_research_units": 1,  # 1 parallel researcher
        "max_researcher_iterations": 2,      # Supervisor can delegate up to 2 times
        "max_react_tool_calls": 3,           # Each researcher can make up to 3 tool calls
        
        # Search configuration
        "search_api": "tavily",  # Using Tavily for web search
        "max_content_length": 50000,
        
        # Thread ID for this conversation
        "thread_id": str(uuid.uuid4())
    }
}

print("✓ Configuration ready")
print(f"  - Research Model: Claude Sonnet 4")
print(f"  - Max Concurrent Researchers: 1")
print(f"  - Max Iterations: 2")
print(f"  - Search API: Tavily")

✓ Configuration ready
  - Research Model: Claude Sonnet 4
  - Max Concurrent Researchers: 1
  - Max Iterations: 2
  - Search API: Tavily


### Execute the Wellness Research

Now let's run the research! We'll ask the system to research evidence-based strategies for improving sleep quality.

The workflow will:
1. **Clarify** - Check if the request is clear (may skip if obvious)
2. **Research Brief** - Transform our request into a structured brief
3. **Supervisor** - Plan research strategy and delegate to researchers
4. **Parallel Research** - Researchers gather information simultaneously
5. **Compression** - Each researcher synthesizes their findings
6. **Final Report** - All findings combined into comprehensive report

In [17]:
# Create our wellness research request
research_request = """
I want to improve my sleep quality. I currently:
- Go to bed at inconsistent times (10pm-1am)
- Use my phone in bed
- Often feel tired in the morning

Please research the best evidence-based strategies for improving sleep quality and create a comprehensive sleep improvement plan for me.
"""

# Execute the graph
async def run_research():
    """Run the research workflow and display results."""
    print("Starting research workflow...\n")
    
    async for event in graph.astream(
        {"messages": [{"role": "user", "content": research_request}]},
        config,
        stream_mode="updates"
    ):
        # Display each step
        for node_name, node_output in event.items():
            print(f"\n{'='*60}")
            print(f"Node: {node_name}")
            print(f"{'='*60}")
            
            if node_name == "clarify_with_user":
                if "messages" in node_output:
                    last_msg = node_output["messages"][-1]
                    print(f"\n{last_msg.content}")
            
            elif node_name == "write_research_brief":
                if "research_brief" in node_output:
                    print(f"\nResearch Brief Generated:")
                    print(f"{node_output['research_brief'][:500]}...")
            
            elif node_name == "supervisor":
                print(f"\nSupervisor planning research strategy...")
                if "supervisor_messages" in node_output:
                    last_msg = node_output["supervisor_messages"][-1]
                    if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                        print(f"Tool calls: {len(last_msg.tool_calls)}")
                        for tc in last_msg.tool_calls:
                            print(f"  - {tc['name']}")
            
            elif node_name == "supervisor_tools":
                print(f"\nExecuting supervisor's tool calls...")
                if "notes" in node_output:
                    print(f"Research notes collected: {len(node_output['notes'])}")
            
            elif node_name == "final_report_generation":
                if "final_report" in node_output:
                    print(f"\n" + "="*60)
                    print("FINAL REPORT GENERATED")
                    print("="*60 + "\n")
                    display(Markdown(node_output["final_report"]))
    
    print("\n" + "="*60)
    print("Research workflow completed!")
    print("="*60)

# Run the research
await run_research()

Starting research workflow...


Node: clarify_with_user

I have sufficient information to proceed with your sleep improvement research. I understand you're looking for evidence-based strategies to address your current sleep challenges: inconsistent bedtimes (10pm-1am), phone use in bed, and morning fatigue. I'll research comprehensive, scientifically-backed sleep hygiene practices and create a personalized sleep improvement plan that addresses these specific issues. Beginning research now.

Node: write_research_brief

Research Brief Generated:
I want to improve my sleep quality based on evidence-based strategies. I currently have three main issues: (1) inconsistent bedtimes ranging from 10pm to 1am, (2) using my phone in bed, and (3) often feeling tired in the morning despite sleeping. Please research comprehensive, scientifically-backed sleep hygiene practices and create a personalized sleep improvement plan that specifically addresses these issues. The research should focus on peer-r


Node: research_supervisor

Node: final_report_generation

FINAL REPORT GENERATED



# Evidence-Based Sleep Improvement Plan: Addressing Inconsistent Bedtimes, Phone Use, and Morning Fatigue

## Understanding Your Sleep Challenges

Your three primary sleep issues—inconsistent bedtimes, phone use in bed, and morning fatigue—are interconnected problems rooted in circadian rhythm disruption and poor sleep hygiene practices. Research shows that sleep consistency refers to maintaining the same bedtime and wake-up time within 30 minutes, including weekends, and this consistency is crucial for health, safety, and performance [1]. A 2023 study found that students with irregular sleep patterns had lower academic performance and more mental health challenges than peers with consistent sleep, even when both groups slept for similar durations [2].

The 3-hour variability in your bedtime (10pm-1am) significantly disrupts your circadian rhythm, which requires daily adjustment since the endogenous human physiological cycle is slightly longer than 24 hours. This coordination is achieved through the suprachiasmatic nucleus (SCN) in the anterior hypothalamus, your body's central pacemaker [3]. When this system is disrupted by irregular timing and blue light exposure from devices, it creates a cascade of sleep problems including difficulty falling asleep, poor sleep quality, and morning fatigue.

## Establishing Consistent Sleep Schedules

### The Science of Sleep Consistency

The circadian photoreceptor system shows peak sensitivity to 450-480 nm light within the blue spectrum, which accounts for blue light's high efficacy in suppressing melatonin and increasing alertness [4]. Your inconsistent bedtime pattern suggests a delayed sleep-wake phase, which affects up to 78% of individuals with sleep timing issues. Research indicates that approximately 40% of people with delayed sleep phase disorder have a family history of circadian rhythm sleep disorders [5].

### Evidence-Based Schedule Fixing Protocol

**Step 1: Establish a Fixed Wake-Up Time First**
Research demonstrates that focusing first on establishing a fixed wake-up time is more effective than trying to control bedtime. Choose a wake-up time that allows for at least 7-9 hours of sleep and maintain it every day, including weekends [6].

**Step 2: Gradual Bedtime Adjustment**
Make gradual adjustments in 15-30 minute increments over several days rather than dramatic changes. If you're currently going to bed at 1am but want to sleep by 10pm, move your bedtime 30 minutes earlier every 2-3 days until you reach your target [6].

**Step 3: Light Exposure Timing**
Light exposure is the most powerful zeitgeber (time cue) for circadian rhythms. Studies show that morning light exposure can advance sleep timing, while evening light delays it. A week of natural light-dark cycle exposure advanced dim light melatonin onset (DLMO) by approximately 2.6 hours in healthy adults [7].

### Chronotherapy and Advanced Timing Interventions

For severe timing issues, chronotherapy involves gradually postponing sleep time by 3 hours every 2-5 days until reaching the desired schedule. However, this requires careful planning and is typically done under medical supervision [5]. A more practical approach combines morning bright light therapy with evening light restriction. Studies using 10,000 lux morning light therapy showed phase advances of 31 minutes in DLMO and 57 minutes in mid-sleep time [7].

## Managing Electronic Device Use Before Bedtime

### The Blue Light Problem

Multiple studies demonstrate that blue light exposure suppresses melatonin secretion and exacerbates sleep problems. Evening exposure to LED-backlit devices can suppress and delay melatonin production, making it significantly harder to fall asleep [4]. A systematic review found that blue light exposure decreased sleep quality in one-fifth of studies and decreased sleep duration in one-third of studies [8].

### Evidence-Based Device Management Strategies

**Complete Device Elimination Protocol**
The most effective approach is complete elimination of screens 1-2 hours before bedtime. Reserve this hour for wind-down activities away from stimulating content [9]. Replace phone use with reading physical books, gentle stretching, meditation, or other relaxing activities.

**Blue Light Filtering Solutions**
If complete elimination isn't immediately feasible, research shows specific interventions can help:

- **Blue Light Blocking Glasses**: A randomized controlled trial found that wearing amber-tinted blue light-blocking lenses for 2 hours before bedtime for 7 consecutive nights significantly improved multiple sleep measures, including sleep quality, duration, soundness, and reduced overall sleep distress [4].

- **Device Settings**: Use "warm" lighting settings on devices. Studies show that "cool" white LED lights caused 12.3% melatonin suppression compared to just 3.6% for "warm" white LEDs. Tunable LED devices can reduce estimated melatonin suppression from 10% at cool settings to just 0.1% at warm settings [10].

**Environmental Light Management**
Research indicates that only brown-tinted blue light filtering solutions proved highly effective, reducing estimated melatonin suppression to below 0.3%. Additionally, bedroom lighting should transition to warm, dim lighting 2-3 hours before bedtime [10].

## Addressing Morning Fatigue and Sleep Quality Optimization

### Understanding Sleep Architecture and Morning Fatigue

Morning fatigue despite adequate sleep duration typically indicates poor sleep quality or disrupted sleep architecture. Sleep architecture consists of 4-6 cycles lasting 90-120 minutes each, including Non-REM sleep (light stages 1-2 and deep slow-wave stage 3) and REM sleep. Research published in Sleep Medicine Reviews demonstrates that sleep quality is just as important as sleep duration for feeling refreshed [11].

### Sleep Inertia and Cortisol Response

Sleep inertia, the grogginess experienced upon waking, was first identified among U.S. Air Force pilots in the 1950s. It's most likely to occur with sudden awakening or insufficient sleep and may last from minutes to over an hour [12]. This is often related to disrupted cortisol awakening response, which should peak within 30-45 minutes after morning awakening and then decline throughout the day.

### Strategies for Reducing Morning Fatigue

**Optimize Sleep Timing**
Waking during lighter sleep stages reduces sleep inertia. While you can't control this precisely without sleep tracking devices, maintaining consistent timing helps your body naturally optimize awakening timing.

**Address Sleep Architecture Issues**
Deep sleep is when your body repairs tissues, strengthens immunity, and consolidates memories. Insufficient deep sleep leads to morning fatigue regardless of total duration [11]. Factors that improve deep sleep include:
- Consistent sleep timing
- Optimal bedroom temperature (65°F/18.3°C)
- Complete darkness
- Minimal noise disruption
- Regular exercise (but not within 3-4 hours of bedtime)

**Morning Light Exposure**
Immediate bright light exposure upon awakening helps establish proper cortisol awakening response and reinforces circadian timing. Studies show this can advance sleep phase and improve morning alertness [7].

## Bedroom Environment Optimization

### Temperature Control

Research consistently shows that people sleep better when their bedroom is optimized for temperature. Any healthy adult experiences a natural drop in body temperature during sleep, which occurs during initial sleep stages because lower core temperature promotes sleepiness. Many experts agree that 65°F (18.3°C) is the ideal bedroom temperature, though this may seem cool, it helps maintain lower core temperature during sleep [13].

### Noise Management

Studies demonstrate that even low-level noise can cause shifts to lighter sleep stages or momentary awakenings. Research examining bedroom environmental factors found that sleep efficiency was significantly related to noise levels (P <.0001), with noise having a -1.85% impact on sleep efficacy after statistical adjustment [14]. Strategies include:
- Use of white noise machines or earplugs
- Heavy curtains or sound-dampening materials
- Addressing sources of intermittent noise

### Light Control

Circadian rhythms are heavily influenced by light and darkness. During the day, sunlight signals the brain to produce cortisol for alertness, while darkness triggers melatonin production for sleepiness. Artificial light exposure in the evening delays circadian rhythms and prolongs sleep onset [13]. Create complete darkness using blackout curtains, eye masks, and elimination of LED indicator lights from electronics.

### Air Quality Considerations

Recent research shows that bedroom air quality significantly affects sleep. In fully adjusted statistical models, sleep efficacy was related to PM2.5 particles (P <.05), CO2 levels (P <.05), and temperature (P <.05). After statistical transformation, sleep efficacy decreased by 2.15% with PM2.5 exposure and 1.73% with temperature increases [14]. Consider air purifiers and adequate ventilation.

## Comprehensive Implementation Plan

### Phase 1: Immediate Changes (Week 1-2)

**Sleep Timing Foundation**
- Choose a fixed wake-up time that allows 7-9 hours of sleep
- Set this wake-up time for every day, including weekends
- Begin moving bedtime 30 minutes earlier every 2-3 days if currently going to bed after 11pm

**Device Management**
- Implement complete phone/device ban in the bedroom
- Charge devices in another room
- Replace bedtime phone use with reading, gentle stretching, or meditation
- If immediate elimination isn't possible, use amber-tinted blue light blocking glasses 2 hours before bedtime

**Environment Setup**
- Set bedroom temperature to 65°F (18.3°C)
- Install blackout curtains or use eye masks
- Remove or cover LED lights from electronics
- Ensure bedroom is as quiet as possible

### Phase 2: Optimization (Week 3-4)

**Light Exposure Protocol**
- Get bright light exposure within 30 minutes of waking (natural sunlight preferred, or 10,000 lux light therapy device)
- Dim all lights to warm settings 2-3 hours before bedtime
- Use only amber or red lighting in evening hours

**Sleep Hygiene Refinement**
- Establish a consistent 1-hour pre-sleep routine including relaxing activities
- Avoid caffeine after 2pm
- Avoid large meals, alcohol, and intense exercise within 3-4 hours of bedtime
- Consider room temperature water beside bed to avoid middle-of-night searches

### Phase 3: Advanced Interventions (Week 5+)

**Circadian Support**
- If sleep timing issues persist, consider low-dose melatonin (0.5-3mg) taken 2-3 hours before desired bedtime, but consult healthcare provider first
- Track sleep patterns to identify optimal timing
- Consider morning exercise to strengthen circadian signals

**Environmental Fine-Tuning**
- Address air quality with purifiers or plants if needed
- Experiment with white noise or earplugs for noise management
- Ensure mattress and pillows support comfortable, uninterrupted sleep

### Monitoring and Adjustment

Track your sleep using a simple log noting bedtime, wake time, sleep quality (1-10 scale), morning fatigue levels, and any deviations from the protocol. Research shows that knowledge alone doesn't translate to improved sleep quality—it requires behavioral change and planned actions for success [15]. Sleep is highly individual, highlighting why precision medicine approaches are key to sleep hygiene success.

Expect gradual improvement over 2-4 weeks rather than immediate results. Studies indicate that circadian rhythm adjustments take time, with some interventions showing effects within days while others require weeks of consistency. The key is maintaining the protocol even when results aren't immediately apparent, as circadian rhythm entrainment is a gradual biological process.

### Sources

[1] The importance of sleep regularity: a consensus statement: https://www.researchgate.net/publication/373741330_The_importance_of_sleep_regularity_a_consensus_statement_of_the_National_Sleep_Foundation_sleep_timing_and_variability_panel

[2] The Science Behind Sleep Consistency and Timing: https://villagedoctor.com/unlock-better-restscience-behind-sleep-consistency/

[3] Treatment of Circadian Rhythm Sleep–Wake Disorders: https://pmc.ncbi.nlm.nih.gov/articles/PMC9886819/

[4] Blocking nocturnal blue light for insomnia: A randomized controlled trial: https://www.sciencedirect.com/science/article/abs/pii/S0022395617308592

[5] Treatment of Circadian Rhythm Sleep–Wake Disorders: https://pmc.ncbi.nlm.nih.gov/articles/PMC9886819/

[6] How to Fix Your Sleep Schedule: https://www.sleepfoundation.org/sleep-hygiene/how-to-reset-your-sleep-routine

[7] ADHD as a circadian rhythm disorder: evidence and implications for chronotherapy: https://www.frontiersin.org/journals/psychiatry/articles/10.3389/fpsyt.2025.1697900/full

[8] The influence of blue light on sleep, performance and wellbeing: https://pmc.ncbi.nlm.nih.gov/articles/PMC9424753/

[9] Sleep hygiene: Simple practices for better rest: https://www.health.harvard.edu/staying-healthy/sleep-hygiene-simple-practices-for-better-rest

[10] The influence of blue light on sleep, performance and wellbeing: https://pmc.ncbi.nlm.nih.gov/articles/PMC9424753/

[11] Why Do I Always Wake Up Tired?: https://www.thesleepreset.com/blog/wake-up-tired

[12] Sleep Inertia: Symptoms, Causes, and More: https://www.verywellhealth.com/how-does-sleep-inertia-make-it-hard-to-wake-up-3014826

[13] Sleep hygiene: Simple practices for better rest: https://www.health.harvard.edu/staying-healthy/sleep-hygiene-simple-practices-for-better-rest

[14] New research on bedroom ventilation and sleep quality: https://www.tandfonline.com/doi/full/10.1080/23744731.2025.2531317

[15] Sleep Hygiene Practices: Where to Now?: https://www.mdpi.com/2673-947X/2/3/13


Research workflow completed!


## Task 9: Understanding the Output

Let's break down what happened:

### Phase 1: Clarification
The system checked if your request was clear. Since you provided specific details about your sleep issues, it likely proceeded without asking clarifying questions.

### Phase 2: Research Brief
Your request was transformed into a detailed research brief that guides the supervisor's delegation strategy.

### Phase 3: Supervisor Delegation
The supervisor analyzed the brief and decided how to break down the research:
- Used `think_tool` to plan strategy
- Called `ConductResearch` to delegate to researchers
- Each delegation specified a focused research topic (e.g., sleep hygiene, circadian rhythm, blue light effects)

### Phase 4: Parallel Research
Researchers worked on their assigned topics:
- Each researcher used web search tools to gather information
- Used `think_tool` to reflect after each search
- Decided when they had enough information
- Compressed their findings into clean summaries

### Phase 5: Final Report
All research findings were synthesized into a comprehensive sleep improvement plan with:
- Well-structured sections
- Evidence-based recommendations
- Practical action items
- Sources for further reading

## Task 10: Key Takeaways & Next Steps

### Architecture Benefits
1. **Dynamic Decomposition** - Research structure emerges from the question, not predefined
2. **Parallel Efficiency** - Multiple researchers work simultaneously
3. **ReAct Quality** - Strategic reflection improves search decisions
4. **Scalability** - Handles token limits gracefully through compression
5. **Flexibility** - Easy to add new tools and capabilities

### When to Use This Pattern
- **Complex research questions** that need multi-angle investigation
- **Comparison tasks** where parallel research on different topics is beneficial
- **Open-ended exploration** where structure should emerge dynamically
- **Time-sensitive research** where parallel execution speeds up results

### When to Use Section-Based Instead
- **Highly structured reports** with predefined format requirements
- **Template-based content** where sections are always the same
- **Sequential dependencies** where later sections depend on earlier ones
- **Budget constraints** where token efficiency is critical

### Extend the System
1. **Add MCP Tools** - Integrate specialized tools for your domain
2. **Custom Prompts** - Modify prompts for specific research types
3. **Different Models** - Try different Claude versions or mix models
4. **Persistence** - Use a real database for checkpointing instead of memory

### Learn More
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [Open Deep Research Repo](https://github.com/langchain-ai/open_deep_research)
- [Anthropic Claude Documentation](https://docs.anthropic.com/)
- [Tavily Search API](https://tavily.com/)

## ❓ Question #3:

What are the trade-offs of using parallel researchers vs. sequential research? When might you choose one approach over the other?

##### Answer:
Trade-offs of Parallel vs Sequential Research:
- Speed vs. Cost/Complexity: Parallel is faster because you can have 5 agents searching 5 different topics at once. The downside occurs in complexity and potential rate-limiting. You burn through tokens much faster in a short burst, which might hit API limits or cost more per minute.
- Context Isolation: In parallel research, Agent A doesn't know what Agent B is finding until the end. In sequential research, Agent B can learn from what Agent A found and adjust its search strategy, which is better for "follow-up" type investigations.

When to choose one over the other:
- Choose Parallel when you have distinct, non-overlapping topics. For example, "Compare the pricing of AWS, Azure, and Google Cloud." You don't need to know AWS's price to look up Azure's. It's distinct lists or comparisons.
- Choose Sequential when the research is a "chain" of dependencies. For example, "Find out who the CEO of Company X is, and then find their recent interviews." You can't do the second part until the first part is done.

## ❓ Question #4:

How would you adapt this deep research architecture for a production wellness application? What additional components would you need?

##### Answer:
To adapt this for a production wellness application, we would need several critical additions beyond the prototype:

- Safety & Medical Disclaimers: Since this is health-related, we need a robust moderation layer. We can't just let the LLM give advice unchecked. We'd need a "Safety Check" node that runs before the final report to ensure no harmful or hallucinations of medical advice are output.
- User Context Integration (The "Wellness Profile"): Instead of just taking a raw query, the system should pull from a user database. If I ask for a diet plan, it should already know my allergies, age, and activity level. This would be a "Context Retrieval" node added to the start of the graph.
- Persistence: The current MemorySaver is in-memory and temporary. For production, we need a lasting database (like Postgres with langgraph-checkpoint-postgres) so users can pause a research task and come back to it days later.
- Caching: Many users will ask similar questions (e.g., "how to sleep better"). We don't want to pay for all the web searches every single time. A semantic cache would check if a similar research report already exists before starting a new one.
- Feedback Loop: We need a way for users to rate the advice. This feedback should be saved and used to fine-tune the supervisor or researcher prompts over time.

## 🏗️ Activity #2: Custom Wellness Research

Using what you've learned, run a custom wellness research task.

**Requirements:**
1. Create a wellness-related research question (exercise, nutrition, stress, etc.)
2. Modify the configuration for your use case
3. Run the research and analyze the output
4. Document what worked well and what could be improved

**Experiment ideas:**
- Research exercise routines for specific conditions (bad knee, lower back pain)
- Compare different stress management techniques
- Investigate nutrition strategies for specific goals
- Explore meditation and mindfulness research

**YOUR CODE HERE**

In [20]:
# Research exercise routines for specific conditions (bad knee)
my_wellness_request = """
I have a bad knee (mild patellofemoral pain syndrome) and want to stay active.
My current situation:
- I used to run 3x per week but had to stop due to knee pain
- I have access to a gym with machines, free weights, and a pool
- My goal is to maintain cardiovascular fitness and leg strength without aggravating my knee

Please research:
1. Evidence-based low-impact cardio alternatives to running (especially for knee-friendly options)
2. Safe leg strengthening exercises that protect the patellofemoral joint
3. Any stretching or mobility routines that help with patellofemoral pain syndrome
"""

# Modified configuration for this use case
my_config = {
    "configurable": {
        "research_model": "anthropic:claude-sonnet-4-20250514",
        "research_model_max_tokens": 10000,
        "compression_model": "anthropic:claude-sonnet-4-20250514",
        "compression_model_max_tokens": 8192,
        "final_report_model": "anthropic:claude-sonnet-4-20250514",
        "final_report_model_max_tokens": 10000,
        "summarization_model": "anthropic:claude-sonnet-4-20250514",
        "summarization_model_max_tokens": 8192,
        "allow_clarification": False,  # Disabled - we provided detailed context
        "max_concurrent_research_units": 2,  # 2 parallel researchers
        "max_researcher_iterations": 3,
        "max_react_tool_calls": 4,
        "search_api": "tavily",
        "max_content_length": 50000,
        "thread_id": str(uuid.uuid4())
    }
}

# Run the research
async def run_custom_research():
    """Run the custom wellness research workflow."""
    print("Starting custom wellness research...\n")
    
    async for event in graph.astream(
        {"messages": [{"role": "user", "content": my_wellness_request}]},
        my_config,
        stream_mode="updates"
    ):
        for node_name, node_output in event.items():
            print(f"\n{'='*60}")
            print(f"Node: {node_name}")
            print(f"{'='*60}")
            
            # Skip if node_output is None
            if node_output is None:
                continue
            
            if node_name == "clarify_with_user":
                if "messages" in node_output and node_output["messages"]:
                    last_msg = node_output["messages"][-1]
                    print(f"\n{last_msg.content}")
            
            elif node_name == "write_research_brief":
                if "research_brief" in node_output and node_output["research_brief"]:
                    print(f"\nResearch Brief Generated:")
                    print(f"{node_output['research_brief'][:500]}...")
            
            elif node_name == "supervisor":
                print(f"\nSupervisor planning research strategy...")
            
            elif node_name == "final_report_generation":
                if "final_report" in node_output and node_output["final_report"]:
                    print(f"\n" + "="*60)
                    print("FINAL REPORT GENERATED")
                    print("="*60 + "\n")
                    display(Markdown(node_output["final_report"]))
    
    print("\n" + "="*60)
    print("Research workflow completed!")
    print("="*60)
await run_custom_research()

Starting custom wellness research...


Node: clarify_with_user

Node: write_research_brief

Research Brief Generated:
I have mild patellofemoral pain syndrome and need to transition from my previous running routine (3x per week) to knee-friendly alternatives while maintaining cardiovascular fitness and leg strength. I have access to a gym with machines, free weights, and a pool. Please research and provide evidence-based recommendations for: (1) Low-impact cardiovascular exercise alternatives to running that are specifically safe for patellofemoral pain syndrome, including detailed guidance on frequency, intens...



Node: research_supervisor

Node: final_report_generation

FINAL REPORT GENERATED



Error generating final report: Error code: 429 - {'type': 'error', 'error': {'type': 'rate_limit_error', 'message': "This request would exceed your organization's rate limit of 30,000 input tokens per minute (org: 9f1a99aa-e76c-4b5d-ac9e-e7690fd731fb, model: claude-sonnet-4-20250514). For details, refer to: https://docs.claude.com/en/api/rate-limits. You can see the response headers for current usage. Please reduce the prompt length or the maximum tokens requested, or try again later. You may also contact sales at https://www.anthropic.com/contact-sales to discuss your options for a rate limit increase."}, 'request_id': 'req_011CXwCQei8FBBjaJfxcetXZ'}


Research workflow completed!


What worked well: The research brief correctly captured my question and the graph ran end-to-end through all nodes.

What could be improved:

- Rate limits: Hit a 429 error and I should have used max_concurrent_research_units: 1 on free tier
- Timeouts: 16 summarization timeouts make me think maybe I should use lower max_content_length next time
- Error handling: Final report just showed raw error instead of returning partial results